## Working with actual hurricane data

In [2]:
# importing everything

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline
import urllib.request
import rasterio

import shapely

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-2e9b0b736433>", line 5, in <module>
    import matplotlib.pyplot as plt
  File "/opt/conda/lib/python3.7/site-packages/matplotlib/__init__.py", line 1015, in <module>
    rcParams = rc_params()
  File "/opt/conda/lib/python3.7/site-packages/matplotlib/__init__.py", line 878, in rc_params
    return rc_params_from_file(matplotlib_fname(), fail_on_error)
  File "/opt/conda/lib/python3.7/site-packages/matplotlib/__init__.py", line 731, in matplotlib_fname
    for fname in gen_candidates():
  File "/opt/conda/lib/python3.7/site-packages/matplotlib/__init__.py", line 720, in gen_candidates
    yield os.path.join(os.getcwd(), 'matplotlibrc')
FileNotFoundError: [Errno 2] No such file or directory

During handling of the above exception, another exception occurred:

Traceback (m

FileNotFoundError: [Errno 2] No such file or directory

In [ ]:
# setting up the figsize so that we don't have to keep entering it when plotting

plt.rcParams['figure.figsize'] = (20, 10)

In [ ]:
post_hurricane_gpd = gpd.read_file('post-event-game-grid.geojson')

In [ ]:
# visualizing the game grid

ax = post_hurricane_gpd.plot()

In [ ]:
# view the columns of this dataframe

post_hurricane_gpd.head(1)

### DEMs

In [ ]:
# bounds of the game grid

northernmost = post_hurricane_gpd['y_centroid'].max() + 0.1
southernmost = post_hurricane_gpd['y_centroid'].min() - 0.1
easternmost = post_hurricane_gpd['x_centroid'].max() + 0.1
westernmost = post_hurricane_gpd['x_centroid'].min() - 0.1
print(northernmost, southernmost, easternmost, westernmost)

In [ ]:
# reading in the set of DEMs for the game grid

filepath = pd.read_csv('post-event-dems.csv')
filepath

In [ ]:
# downloading each DEM

for idx, path in enumerate(filepath['downloadURL']):
    urllib.request.urlretrieve(path, 'post-event-images/dem-orig-{}.tif'.format(idx))
print('Done!')

In [ ]:
# creating each elevation map from the DEMs

!/opt/conda/bin/gdaldem hillshade post-event-images/dem-orig-0.tif post-event-images/dem-hs-0.tif

!/opt/conda/bin/gdaldem hillshade post-event-images/dem-orig-1.tif post-event-images/dem-hs-1.tif

!/opt/conda/bin/gdaldem hillshade post-event-images/dem-orig-2.tif post-event-images/dem-hs-2.tif

!/opt/conda/bin/gdaldem hillshade post-event-images/dem-orig-3.tif post-event-images/dem-hs-3.tif

!/opt/conda/bin/gdaldem hillshade post-event-images/dem-orig-4.tif post-event-images/dem-hs-4.tif

!/opt/conda/bin/gdaldem hillshade post-event-images/dem-orig-5.tif post-event-images/dem-hs-5.tif

!/opt/conda/bin/gdaldem hillshade post-event-images/dem-orig-6.tif post-event-images/dem-hs-6.tif

!/opt/conda/bin/gdaldem hillshade post-event-images/dem-orig-7.tif post-event-images/dem-hs-7.tif

In [ ]:
# visualizing one of the elevation maps (sanity check)
# DOESN'T WORK

with rasterio.open('post-event-images/dem-hs-0.tif', 'r') as src:
    print(src.meta)
    hillshade = src.read(1)
    hillshade_mask = src.read_masks(1)
    hillshade_transform = src.profile['transform']
    
    fig, ax = plt.subplots(1,figsize = (10,10))
    
    rasterio.plot.show(np.ma.masked_where(hillshade_mask==0, hillshade), transform=hillshade_transform, ax=ax)
   
    plt.show() 

### OSM